In [1]:
import sys, os
sys.path.insert(1, '/nfs/cms/martialc/DTUpgrade/src')

In [2]:
%load_ext autoreload
%autoreload 2

import dtupy_analysis
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyarrow as pa
import pyarrow.parquet as pq
import dask

import dtupy_analysis.dqm as dqm

In [51]:
file = '/nfs/cms/martialc/DTUpgrade/data/sxa5_240802_full.parquet'
mudata= dqm.MuFile.to_dask(file)

TypeError: 'NoneType' object is not subscriptable

In [53]:
mudata[mudata.sl == 1]

,bx,cell,channel,index_t,layer,link,obdt_ctr,obdt_type,sl,station,tdc
0,1005,36,204,26,2,9,9,0,1,2,11
2,1005,1,215,31,4,9,4,0,1,1,13
3,1005,46,95,36,3,9,11,0,1,2,11
4,1005,33,219,37,3,9,8,0,1,2,12
6,1005,7,223,39,3,9,5,0,1,1,16
...,...,...,...,...,...,...,...,...,...,...,...
99989,1005,43,142,278137,2,9,10,0,1,2,10
99990,1005,35,62,278138,4,9,8,0,1,2,10
99991,1005,39,162,278145,4,9,9,0,1,2,11
99996,1005,10,38,278156,4,9,6,0,1,1,14
